In [1]:
from google.colab import drive
drive.mount('drive')
import pandas as pd
import numpy as np
diabetes_df = pd.read_csv('/content/drive/MyDrive/CL/Day 4/calories_exercise_data.csv', sep=',' )

diabetes_df

Mounted at drive


,user_id,calories,gender,age,height,weight,duration,heart_rate,body_temp
0,14733363,231.0,male,68,190.0,94.0,29.0,105.0,40.8
1,14861698,66.0,female,20,166.0,60.0,14.0,94.0,40.3
2,11179863,26.0,male,69,179.0,79.0,5.0,88.0,38.7
3,16180408,71.0,female,34,179.0,71.0,13.0,100.0,40.5
4,17771927,35.0,female,27,154.0,58.0,10.0,81.0,39.8
...,...,...,...,...,...,...,...,...,...
14995,15644082,45.0,female,20,193.0,86.0,11.0,92.0,40.4
14996,17212577,23.0,female,27,165.0,65.0,6.0,85.0,39.2
14997,17271188,75.0,female,43,159.0,58.0,16.0,90.0,40.1
14998,18643037,11.0,male,78,193.0,97.0,2.0,84.0,38.3


In [3]:
!pip install ydata_profiling

import ydata_profiling as ydp

diabetes_report = ydp.ProfileReport(diabetes_df)
diabetes_report

Output hidden; open in https://colab.research.google.com to view.

In [5]:
diabetes_df.drop(columns=['user_id'],inplace=True,axis=1)
diabetes_df

In [6]:
diabetes_df

,calories,gender,age,height,weight,duration,heart_rate,body_temp
0,231.0,male,68,190.0,94.0,29.0,105.0,40.8
1,66.0,female,20,166.0,60.0,14.0,94.0,40.3
2,26.0,male,69,179.0,79.0,5.0,88.0,38.7
3,71.0,female,34,179.0,71.0,13.0,100.0,40.5
4,35.0,female,27,154.0,58.0,10.0,81.0,39.8
...,...,...,...,...,...,...,...,...
14995,45.0,female,20,193.0,86.0,11.0,92.0,40.4
14996,23.0,female,27,165.0,65.0,6.0,85.0,39.2
14997,75.0,female,43,159.0,58.0,16.0,90.0,40.1
14998,11.0,male,78,193.0,97.0,2.0,84.0,38.3


In [9]:
diabetes_df = pd.get_dummies(diabetes_df, columns=['gender'], drop_first=True)


In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
diabetes_df[['age', 'height', 'duration', 'heart_rate', 'body_temp']] = scaler.fit_transform(diabetes_df[['age', 'height', 'duration', 'heart_rate', 'body_temp']])


In [13]:
from sklearn.model_selection import train_test_split

# Podział danych na zmienne niezależne (X) i zależną (y)
X = diabetes_df.drop(columns=['calories'])
y = diabetes_df['calories']

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error

# Zdefiniowanie modelu
xgb_regressor = xgb.XGBRegressor(random_state=42, n_jobs=-1)

# Definiowanie siatki hiperparametrów
params = {
    'max_depth': [3, 5, 7, 10],
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.01, 0.1, 0.2],
    'gamma': [0, 0.1, 0.3, 0.5]
}

# Randomized Search
random_search = RandomizedSearchCV(estimator=xgb_regressor, param_distributions=params, n_iter=50, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error', random_state=42)

# Dopasowanie modelu
random_search.fit(X_train, y_train)

# Najlepsze parametry
best_params = random_search.best_params_
print("Best Parameters:", best_params)

# Trenowanie modelu z najlepszymi parametrami
best_xgb_regressor = xgb.XGBRegressor(**best_params, random_state=42, n_jobs=-1)
best_xgb_regressor.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best Parameters: {'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0.3}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.3, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

In [16]:
# Dokonanie predykcji na zbiorze testowym
y_pred = best_xgb_regressor.predict(X_test)

# Obliczenie średniego błędu bezwzględnego
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 1.0957343979014291
